# Battle of Neighbourhood Project

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
import requests
from geopy.geocoders import Nominatim
print ('done!')


done!


In [2]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

In [3]:

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                       

In [5]:
import requests # library to handle requests
from bs4 import BeautifulSoup
import pandas as pd

In [7]:
#getting neighborhood from scraping "healthysf organisation website 
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])

In [8]:
#locking the neighborhood names
df.columns = df.iloc[0]
df = df.iloc[1:-1, :-1]
sf_data = df
sf_data.head()

,Zip Code,Neighborhood
1,94102,Hayes Valley/Tenderloin/North of Market
2,94103,South of Market
3,94107,Potrero Hill
4,94108,Chinatown
5,94109,Polk/Russian Hill (Nob Hill)


In [10]:
sf_data.shape

(21, 2)

In [12]:
#convert addresses of neighborhoods to lat long

!pip install uszipcode
from uszipcode import SearchEngine

search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

sf_data["Latitude"] = latitude
sf_data["Longitude"] = longitude

sf_data.head()

,Zip Code,Neighborhood,Latitude,Longitude
1,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420
2,94103,South of Market,37.780,-122.410
3,94107,Potrero Hill,37.770,-122.390
4,94108,Chinatown,37.791,-122.409
5,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420


In [13]:
#finding the lat lot of San francisco

address = 'San Francisco'

geolocator = Nominatim(user_agent = "san_francisco_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of San Francisco are 37.7790262, -122.4199061.


In [14]:
#Mapping neighborhoods

map_sf = folium.Map(location = [latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(sf_data['Latitude'], sf_data['Longitude'], sf_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_sf)  
    
map_sf

In [72]:
#initiating Foursquare 
CLIENT_ID = 'NWFVOK5MLY1N0HY13EOSQ1YMLZ0AYPRQZG3IJIFXJ2GCCV4P' # your Foursquare ID
CLIENT_SECRET = 'O2WBUG5541WC0MR0W4QVJEAVQYLGPA5C4XEZMXX4W1SBUK1E' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100


In [47]:
#Get nearby venues to neigborhoods lat lon- radius 500 metres

def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)

In [ ]:
sf_venues = getNearbyVenues(names = sf_data['Neighborhood'],
                                   latitudes = sf_data['Latitude'],
                                   longitudes = sf_data['Longitude']
                                  )
                                  


In [27]:
sf_venues.columns


Index(['Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude',
       'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category'],
      dtype='object')

In [31]:
#indexing the dataset
sf_venues_idx = sf_venues.set_index('Neighborhood')
sf_venues_idx

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,War Memorial Opera House,37.778601,-122.420816,Opera House
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,San Francisco Ballet,37.778580,-122.420798,Dance Studio
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Herbst Theater,37.779548,-122.420953,Concert Hall
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,War Memorial Court,37.779042,-122.420971,Park
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,"Books, Inc.",37.781614,-122.420531,Bookstore
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,PLÄJ,37.778733,-122.422106,Scandinavian Restaurant
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Main Stage Of Davies Symphony Hall,37.777703,-122.420476,Performing Arts Venue
Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Whitechapel,37.782230,-122.418884,Cocktail Bar


In [23]:
sf_venues.shape

(983, 7)

In [49]:
#look at number of gyms in each neighborhood
df1 = sf_venues.loc[sf_venues['Venue Category'].str.contains('Gym') == True]
df1.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
126,South of Market,37.78,-122.41,LaLanne Fitness CrossFit,37.780912,-122.406661,Gym / Fitness Center
139,South of Market,37.78,-122.41,Ralph Gracie Jiu-Jitsu,37.776686,-122.411155,Gym
150,South of Market,37.78,-122.41,Raven Fitness,37.776526,-122.407658,Gymnastics Gym
182,Potrero Hill,37.77,-122.39,UCSF Bakar Fitness & Rec Center,37.768146,-122.393290,Gym
224,Potrero Hill,37.77,-122.39,Madrone Gym,37.771969,-122.389431,Gym


In [36]:
df1.shape

(36, 7)

In [37]:
df1.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Castro/Noe Valley,1,1,1,1,1,1
Chinatown,5,5,5,5,5,5
Haight-Ashbury,1,1,1,1,1,1
Inner Mission/Bernal Heights,2,2,2,2,2,2
Lake Merced,2,2,2,2,2,2
Marina,5,5,5,5,5,5
North Beach/Chinatown,5,5,5,5,5,5
Polk/Russian Hill (Nob Hill),4,4,4,4,4,4
Potrero Hill,6,6,6,6,6,6


In [43]:
#Juice Bars in neigborhoods
df2 = sf_venues.loc[sf_venues['Venue Category'].str.contains('Juice') == True]
df2.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
45,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Project Juice,37.777035,-122.422375,Juice Bar
217,Potrero Hill,37.77,-122.39,JOE & THE JUICE,37.773257,-122.391227,Juice Bar
540,Castro/Noe Valley,37.76,-122.44,Project Juice,37.760716,-122.435023,Juice Bar
832,Marina,37.80,-122.44,Project Juice,37.800367,-122.440067,Juice Bar
901,Lake Merced,37.72,-122.48,Natural Sensations,37.722021,-122.478612,Juice Bar


In [44]:
df2.shape

(6, 7)

In [40]:
df2.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Castro/Noe Valley,1,1,1,1,1,1
Hayes Valley/Tenderloin/North of Market,1,1,1,1,1,1
Lake Merced,1,1,1,1,1,1
Marina,1,1,1,1,1,1
North Beach/Chinatown,1,1,1,1,1,1
Potrero Hill,1,1,1,1,1,1


In [46]:
df3 = sf_venues.loc[sf_venues['Venue Category'].str.contains('Vegetarian') == True]
df3.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
32,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Ananda Fuara,37.777693,-122.416353,Vegetarian / Vegan Restaurant
73,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Thai Idea Vegetarian Restaurant,37.783395,-122.419141,Vegetarian / Vegan Restaurant


In [61]:
df1_idx = df1.set_index('Neighborhood')
df2_idx = df2.set_index('Neighborhood')
df3_idx = df3.set_index('Neighborhood')

In [66]:
dfinterim = pd.merge(df1, df2, how='outer')
dftotal= pd.merge(dfinterim,df3, how='outer')
dftotal.shape

(44, 7)

In [67]:
dftotal.head(10)

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,South of Market,37.780,-122.410,LaLanne Fitness CrossFit,37.780912,-122.406661,Gym / Fitness Center
1,South of Market,37.780,-122.410,Ralph Gracie Jiu-Jitsu,37.776686,-122.411155,Gym
2,South of Market,37.780,-122.410,Raven Fitness,37.776526,-122.407658,Gymnastics Gym
3,Potrero Hill,37.770,-122.390,UCSF Bakar Fitness & Rec Center,37.768146,-122.393290,Gym
4,Potrero Hill,37.770,-122.390,Madrone Gym,37.771969,-122.389431,Gym
5,Potrero Hill,37.770,-122.390,Parklab Fitness Court,37.770659,-122.393028,Gym / Fitness Center
6,Potrero Hill,37.770,-122.390,PowerPlay SF,37.772390,-122.390816,Gym / Fitness Center
7,Potrero Hill,37.770,-122.390,MB360 Gym,37.772488,-122.391838,Gym
8,Potrero Hill,37.770,-122.390,Azure Apartments Gym,37.772844,-122.392893,Gym
9,Chinatown,37.791,-122.409,The Olympic Club,37.788181,-122.411067,Gym / Fitness Center


In [86]:
# create map of Bay Area using latitude and longitude values
map_sv = folium.Map(location=[37.7749, -122.4194], zoom_start=12)

# add markers to map
for lat, lng, venue, neighbourhood in zip(df1['Neighborhood Latitude'],df1['Neighborhood Longitude'],df1['Neighborhood'],df1['Venue'], ):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='green',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sv) 
for lat, lng, venue, neighbourhood in zip(df2['Neighborhood Latitude'],df2['Neighborhood Longitude'],df2['Neighborhood'],df1['Venue'], ):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sv) 
for lat, lng, venue, neighbourhood in zip(df3['Neighborhood Latitude'],df3['Neighborhood Longitude'],df3['Neighborhood'],df3['Venue'], ):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='purple',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_sv) 
map_sv

In [ ]:
# there is a great scope to open a new juice bar in the different neighborhoods. Based on the 500 radius Chinatown does not have any juice bars and it has 5 gym venues. Therefore this would be one of the suggested locations for a new business venture